# On The Free Energy Penalty of Cavity Formation In Salt Solutions: Rethinking the Terms “Kosmotropic” and “Chaotropic”.

Stefan Hervø-Hansen<sup>a,*</sup> and Nobuyuki Matubayasi<sup>a,*</sup>.<br><br>
<sup>a</sup> Division of Chemical Engineering, Graduate School of Engineering Science, Osaka University, Toyonaka, Osaka 560-8531, Japan.<br>
<sup>*</sup> To whom correspondence may be addressed: stefan@cheng.es.osaka-u.ac.jp and nobuyuki@cheng.es.osaka-u.ac.jp.

## Analysis & Model Construction


## Import of Python Modules & Auxiliary Functions

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import mdtraj as md
import parmed as pmd
from math import isclose

In [ ]:
def MC_CalcVolume(traj, top, Niterations, molecule_box_ratio=0, blockaverages=None, debugging=False):
    """
    Computes the molecular volume by the usage of a Hit & Miss Monte Carlo algorithm.

    This function simply wraps the ``+`` operator, and does not
    do anything interesting, except for illustrating what
    the docstring of a very simple function looks like.

    Parameters
    ----------
    traj : MDTraj trajectory
        MD/MC trajectory loaded using the MD traj library.
    top : parmed topology
        topology containing the LJ parameters for all particles in the trajectory.
    molecule_box_ratio: float, default=0
        Extra lengths added to box enclosing molecule.
    blockaverages: int>1 or None, default=None
        Computation of N>1 bloc kaverages, if None this is skipped.
    debugging: bool, defualt=False
        Returning volume of molecule, volume of box, and Hit&Miss ratio. 
    
    Returns
    -------
    np.array
        The volumes of the molecules for the various frames.
        If debugging == True, the volumes are followed by the box volume and Hit&Miss ratio for the last frame.
    """
    
    # Find all radii for solute molecule
    atomic_radii = np.empty(shape=(traj.n_atoms)) # In nanometer
    for atom in top.atoms:
        atomic_radii[atom.idx] = 0.5*atom.sigma   # In Ångstrom
        atomic_radii[atom.idx] /= 10              # Convert to nanometer
        
    # Initialize data collection and block averages
    if blockaverages:
        if blockaverages < 2:
            raise Exception('blockaverages must be larger or equal to 2 or None.')
        if not isinstance(blockaverages, int):
            raise Exception('blockaverages must be an int.')
        Vdata = np.zeros(shape=(traj.n_frames, blockaverages))
    else:
        Vdata = np.zeros(shape=(traj.n_frames))
    
    
    for frameNr, frame in enumerate(traj): # Trajectory loop
        # Find min/max molecule dimensions and from there box dimensions
        molecule_dimensions = np.zeros(shape=(2,3))
        for i in range(molecule_dimensions.shape[0]):
            for j in range(molecule_dimensions.shape[1]):
                if i == 0:
                    molecule_dimensions[i][j] = np.min(frame.xyz[0][:,j] - atomic_radii)
                else:
                    molecule_dimensions[i][j] = np.max(frame.xyz[0][:,j] + atomic_radii)
        box_dimensions = molecule_dimensions + molecule_dimensions * molecule_box_ratio
        
        # Calculate box volume
        Vbox = (np.linalg.norm(box_dimensions[0][0] - box_dimensions[1][0])* # x
                np.linalg.norm(box_dimensions[0][1] - box_dimensions[1][1])* # y
                np.linalg.norm(box_dimensions[0][2] - box_dimensions[1][2])) # z
        
        if not blockaverages:
            Nblockaverages = 1
        else:
            Nblockaverages = blockaverages
            
        for blockaverage in range(Nblockaverages): # Block average loop
            Nhits = 0
            for i in range(Niterations): # MC loop
                # Generate random point
                point = np.array([np.random.uniform(box_dimensions[0][0], box_dimensions[1][0]),  # x
                                  np.random.uniform(box_dimensions[0][1], box_dimensions[1][1]),  # y
                                  np.random.uniform(box_dimensions[0][2], box_dimensions[1][2])]) # z
                
                # Check if point is found within the vdw spheres of particles
                for atom_idx, atom_coordinate in enumerate(frame.xyz[0]): # Particle loop
                    atom_radius = atomic_radii[atom_idx]
                    distance = np.linalg.norm(atom_coordinate - point)
                    if distance <= atom_radius:
                        Nhits += 1
                        break
                        
            # Calculate the volume: V = n_hit / n_total * V_box
            if blockaverages:
                Vdata[frameNr, blockaverage] = Nhits / Niterations * Vbox
            else:
                Vdata[frameNr] = Nhits / Niterations * Vbox
    
    if debugging:
        return Vdata, Vbox, Nhits/Niterations
    else:
        return Vdata

In [ ]:
# TESTING ALGORITHM #
# The MC algorithm is tested on water that only possess LJ parameters for oyxgen thus being a spherical particle.
# The analytical solutions for r being the radius of the sphere:
# Volume of sphere: 4/3*π*r^3
# Volume of box inscribing sphere: (2*r)^3
# Ratio of box volume to sphere volume: 6/π.

topology = pmd.load_file('Force_fields/OPC.itp', xyz='PDB_files/OPC.pdb')
water_pdb = md.load_pdb('PDB_files/OPC.pdb')
water_radius = topology.atoms[0].sigma*0.5*0.1 # in nanometer
V_molecule_analytical = 4/3*np.pi*water_radius**3
V_box_analytical = (2*water_radius)**3
ratio = np.pi/6
Vmolecule, Vbox, sampled_ratio = MC_CalcVolume(water_pdb, topology, Niterations=1000000, blockaverages=None, debugging=True)

assert isclose(V_molecule_analytical, Vmolecule[0], rel_tol=1e-3), 'Molecular volume is not within the tolerance.'
assert isclose(V_box_analytical, Vbox, rel_tol=1e-7), 'Box volume is not within tolerance.'
assert isclose(ratio, sampled_ratio, rel_tol=1e-3), 'Box volume to spherical volume is not within tolerance.'

# RESEARCH NOTES
_will be deleted when the study is finished_